In [1]:
import pandas as pd

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout


2024-12-02 09:51:40.728520: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-02 09:51:40.844414: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-02 09:51:40.935497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733113301.025577    7152 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733113301.052657    7152 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 09:51:41.270297: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
X_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_train.csv", header=None)
X_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_test.csv", header=None)
y_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_train.csv", header=None)
y_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_test.csv", header=None)



print("X train ",X_train.shape)
print("X test",X_test.shape)
print("Y_Train", y_train.shape)
print("Y_test", y_test.shape)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

X train  (1281, 1281)
X test (375, 1281)
Y_Train (1281, 1)
Y_test (375, 1)


In [5]:
model = Sequential()

model.add(Dense(128, activation='relu', input_dim=1000))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

/home/pavithra/projects/spam_or_ham_fsec/spam_ham_env/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-12-02 03:05:01.835854: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       128,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 136,449 (533.00 KB)

 Trainable params: 136,449 (533.00 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
import keras
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
)



def fit_model(model, epochs, X_train, y_train,
              X_test, y_test, ):
    class_weight = {0: 1., 1: 10.}

    history = model.fit(X_train, y_train,
                validation_data=(X_test, y_test),
                epochs=epochs,
                batch_size=64,
                verbose=1,
                class_weight=class_weight)

    return history


def get_classification_metrics(y_test: pd.DataFrame, y_pred: pd.DataFrame) -> dict[str, str]:
    """
    Returns the classification report as a dictionary.
    """
    return {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred),
        "confusion_matrix": confusion_matrix(y_test, y_pred),
        "fb_score": fbeta_score(y_test, y_pred, beta=0.2),
    }


from tensorflow.keras import backend as K

def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_true = K.cast(y_true, K.floatx())
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

In [11]:
def create_model():
    """
    Create and return the model I wanna try on.
    Which is a nueral network with one hidden layer and a dropout layer.
    """
    model = Sequential()

    model.add(LSTM(256, return_sequences=True, input_shape=(1281, 1)))
    model.add(Dropout(0.2))

    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    return model

model = create_model()

model.compile(optimizer=keras.optimizers.Adam(),
                  #loss=keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'],
                  loss=focal_loss(gamma=2., alpha=0.25))

history = fit_model(model, 6, X_train, y_train, X_test, y_test)


/home/pavithra/projects/spam_or_ham_fsec/spam_ham_env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 1281, 256)      │       264,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 1281, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 1281, 128)      │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 1281, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 512,833 (1.96 MB)

 Trainable params: 512,833 (1.96 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/6
21/21 ━━━━━━━━━━━━━━━━━━━━ 107s 5s/step - accuracy: 0.4959 - loss: 0.1163 - val_accuracy: 0.1013 - val_loss: 4.9137e-08
Epoch 2/6
21/21 ━━━━━━━━━━━━━━━━━━━━ 107s 5s/step - accuracy: 0.4669 - loss: 1.6952e-04 - val_accuracy: 0.1013 - val_loss: 5.3236e-09
Epoch 3/6
21/21 ━━━━━━━━━━━━━━━━━━━━ 124s 6s/step - accuracy: 0.4661 - loss: 2.3068e-05 - val_accuracy: 0.1013 - val_loss: 2.7111e-09
Epoch 4/6
21/21 ━━━━━━━━━━━━━━━━━━━━ 108s 5s/step - accuracy: 0.4838 - loss: 4.5643e-05 - val_accuracy: 0.1013 - val_loss: 1.7599e-09
Epoch 5/6
21/21 ━━━━━━━━━━━━━━━━━━━━ 101s 5s/step - accuracy: 0.4764 - loss: 2.0707e-05 - val_accuracy: 0.1013 - val_loss: 1.1879e-09
Epoch 6/6
21/21 ━━━━━━━━━━━━━━━━━━━━ 110s 5s/step - accuracy: 0.4577 - loss: 1.1885e-05 - val_accuracy: 0.1013 - val_loss: 8.3066e-10


In [12]:
y_pred = model.predict(X_test)

12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step


In [13]:
y_pred_binary = (y_pred >= 0.5).astype(int)


In [14]:
get_classification_metrics(y_test, y_pred_binary)

{'accuracy': 0.10133333333333333,
 'precision': np.float64(0.10133333333333333),
 'recall': np.float64(1.0),
 'f1_score': np.float64(0.18401937046004843),
 'confusion_matrix': array([[  0, 337],
        [  0,  38]]),
 'fb_score': np.float64(0.10496122383937109)}

In [9]:
model.save("lstm.h5")

In [28]:
import dagshub
import mlflow
dagshub.init(repo_owner='pavipd495', repo_name='spam_or_ham_fsec', mlflow=True)
mlflow.set_experiment("Spam_Or_Ham")


with mlflow.start_run(run_name="dl_lstm"):

    # Get the predictions.
    y_pred = model.predict(X_test)
    y_pred = (y_pred >= 0.5).astype(int)

    # Log metrics to mlflow.
    metrics = get_classification_metrics(y_test=y_test, y_pred=y_pred)
    mlflow.log_metric("accuracy", metrics["accuracy"],)
    mlflow.log_metric("precision", metrics["precision"])
    mlflow.log_metric("recall", metrics["recall"])
    mlflow.log_metric("f1_score", metrics["f1_score"])

    mlflow.keras.log_model(model, "lstm_us_0_9_os_02")

Accessing as pavipd495

Initialized MLflow to track repo "pavipd495/spam_or_ham_fsec"

Repository pavipd495/spam_or_ham_fsec initialized!

12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 462ms/step
🏃 View run dl_lstm at: https://dagshub.com/pavipd495/spam_or_ham_fsec.mlflow/#/experiments/2/runs/70278634e1fa412e89c661329ac015c0
🧪 View experiment at: https://dagshub.com/pavipd495/spam_or_ham_fsec.mlflow/#/experiments/2
